In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from pathlib import Path

from ciflows.lightning import plApproximateFlowModel
from torchvision.utils import make_grid, save_image
import matplotlib.pyplot as plt

/Users/adam2392/miniforge3/envs/ciflows/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
# reload the model from checkpoint
root = Path('/Users/adam2392/pytorch_data/ciflows/results/')

model_dir = root / 'check_injflow_mnist_v2'
epoch=999
step=430000

# model_dir = root / 'check_glowflow_mnist_v1'
# epoch=99
# step=43000

model_dir = root / 'check_fff_mnist_v1'
epoch=999
step=215000
model_fname = model_dir / f'epoch={epoch}-step={step}.ckpt'

model = plApproximateFlowModel.load_from_checkpoint(model_fname)

# model = plFlowModel.load_from_checkpoint(model_fname)

# model_fname = model_dir / 'check_injflow_mnist_v1-model.pt_final.pt'
# model = torch.load(model_fname, map_location='mps')

/Users/adam2392/miniforge3/envs/ciflows/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'latent' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['latent'])`.
/Users/adam2392/miniforge3/envs/ciflows/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'encoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['encoder'])`.
/Users/adam2392/miniforge3/envs/ciflows/lib/python3.11/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'decoder' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['decoder'])`.


In [25]:
latent = model.latent

print(latent)
samples = latent.sample(16)
print(samples.shape)
print(model.decoder.embed_dim)
self = model.decoder
print(self.patch_size * self.patch_size * self.in_channels)

DiagGaussian()
torch.Size([16, 64])
64
16


In [26]:

# Step 7: Evaluation
# model.eval()  # Set model to evaluation mode
# with torch.no_grad():
#     loss = evaluate_loss(model, val_loader, device) # Compute loss
#     avg_test_loss = loss.item()  # Accumulate test loss

# print(f'Test Loss: {avg_test_loss:.4f}')

# Step 8: Generate Samples
num_samples = 16
model.eval()
model = model.to(torch.device('cpu'))
with torch.no_grad():
    samples, _ = model.sample(num_samples)  # Generate samples
    samples = samples.view(-1, 1, 28, 28).cpu()  # Reshape for plotting

# Visualize generated samples

# Visualize generated samples
grid_size = make_grid(samples, nrow=4, padding=2)
save_image(grid_size, 'generated_samples.png', normalize=True)

print(grid_size.shape)
fig, axes = plt.subplots(4, 4, figsize=(8, 8))
for i in range(num_samples):
    ax = axes[i // 4, i % 4]
    ax.imshow(samples[i].squeeze(), cmap='gray')
    ax.axis('off')

plt.show()

ValueError: not enough values to unpack (expected 3, got 2)

In [35]:
num_samples = 2

z, log_q = model.model.q0(num_samples)
for flow in model.model.flows:
    z, log_det = flow(z)
    log_q -= log_det
    print()
    print(flow, z.flatten()[0], log_q[0])
return z, log_q


GlowBlock(
  (flows): ModuleList(
    (0): AffineCouplingBlock(
      (flows): ModuleList(
        (0): Split()
        (1): AffineCoupling(
          (param_map): ConvNet2d(
            (net): Sequential(
              (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
              (1): LeakyReLU(negative_slope=0.0)
              (2): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
              (3): LeakyReLU(negative_slope=0.0)
              (4): Conv2d(64, 2, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            )
          )
        )
        (2): Merge()
      )
    )
    (1): Invertible1x1Conv()
    (2): ActNorm()
  )
) tensor(1.1539, grad_fn=<SelectBackward0>) tensor(-152.7195, grad_fn=<SelectBackward0>)

GlowBlock(
  (flows): ModuleList(
    (0): AffineCouplingBlock(
      (flows): ModuleList(
        (0): Split()
        (1): AffineCoupling(
          (param_map): ConvNet2d(
            (net): Sequential(
              (0): Conv2d(1, 64, kernel

SyntaxError: 'return' outside function (1581086800.py, line 9)